In [ ]:

print("hello")

hello


In [1]:
import requests
import httpx 

In [8]:
path = "/healthz"
base_url = "http://localhost:8002"
endpoint = f"{base_url}{path}"
endpoint

'http://localhost:8002/healthz'